# 🛡️ Interactive Squad Simulation
Follow the directions at the top of each cell. After running the last cell, use the sliders and dropdowns below to run and visualize the simulation.

Colab Environment

In [ ]:
# If you are working in colab, use this block to clone the GitHub repo, install dependencies, and pull down the latest changes.
%cd /content/
!rm -rf SE3250-Spring2025-SquadSimulation
!git clone https://github.com/SuprMunchkin/SE3250-Spring2025-SquadSimulation.git
%cd SE3250-Spring2025-SquadSimulation
%pip install -r requirements.txt

!git fetch origin
!git checkout "24-create-monte-carlo-front-end" #Change this line to use your branch
%ls

VS Code Environment

In [3]:
%pip install -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import and setup
import sys
import yaml
import matplotlib.pyplot as plt
from ipywidgets import interact

# Custom imports
import os
sys.path.append("../models")
from models.squad_simulation import run_simulation
yaml_path = "config/simulation.yaml"
with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)

map_size = config['map_size']

# Define interactive runner
def run_interactive_sim(blue_stock, red_stock, direction_deviation, armor_type, environment):
    """
    Run the interactive squad simulation and plot the results.

    Parameters:
        blue_stock (int): Number of blue units.
        red_stock (int): Number of red units.
        direction_deviation (int): Direction deviation in degrees.
        armor_type (str): Type of armor for blue units.
        environment (str): Simulation environment.
    """
    params = {
        "blue_stock": blue_stock,
        "red_stock": red_stock,
        "direction_deviation": direction_deviation,
        "armor_type": armor_type,
        "environment": environment
    }
    result = run_simulation(params, full_log=True)

    blue_positions = result['blue_positions']
    red_position = result['red']['current_position']

    plt.figure(figsize=(8, 8))

    # Plot Blue Patrol Path
    if blue_positions:  # Check if there are any blue positions
        x_vals, y_vals = zip(*blue_positions)
        plt.plot(x_vals, y_vals, label='Blue Patrol Path', color='blue')
        plt.scatter(x_vals[0], y_vals[0], c='green', label='Start', zorder=5)
        plt.scatter(x_vals[-1], y_vals[-1], c='purple', label='End', zorder=5)

    # Plot red Position(s)
    if red_position: # Check if red_positions exist
        # Check if it's a list of positions (moving) or a single position (stationary)
        if isinstance(red_position[0], (list, tuple)):
            # It's a list of positions, plot the path
            hx_vals, hy_vals = zip(*red_position)
            plt.plot(hx_vals, hy_vals, label='red Path', linestyle='--', color='red')
        else:
            # It's a single position, plot a scatter point
            hx, hy = red_position # Unpack the single coordinate pair
            plt.scatter(hx, hy, c='red', label='red Position', zorder=5)
    else:
        print("Warning: No red position data available.")

    plt.xlim(0, map_size)
    plt.ylim(0, map_size)
    plt.title("Squad Movement Simulation")
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

    print(f"👥 Blue Remaining: {result['blue']['stock']} / {params['blue_stock']}")
    print(f"🔴 red Remaining: {result['red']['stock']} / {params['red_stock']}")

    # Print statements here can be used for troublshooting.
    #print("Blue positions: ", positions)
    #print("red positions: ", red_positions)

# Create interactive widget interface
interact(
    run_interactive_sim,
    blue_stock=(1, 20),
    red_stock=(1, 40),
    direction_deviation=(0, 45, 5),
    armor_type=list(config['armor_profiles'].keys()),
    environment=list(config['threat_probs'].keys())
)

interactive(children=(IntSlider(value=10, description='blue_stock', max=20, min=1), IntSlider(value=20, descri…

<function __main__.run_interactive_sim(blue_stock, red_stock, direction_deviation, armor_type, environment)>

In [4]:
from ipywidgets import Button, VBox, Output
import itertools
import pandas as pd
import sys
import yaml
import matplotlib.pyplot as plt
from IPython.display import display # Import display
from tqdm.notebook import tqdm # Import tqdm for notebooks

# Custom imports
import os
sys.path.append("../models")
from models.squad_simulation import run_simulation
yaml_path = "config/simulation.yaml"
with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)

# Define the number of simulations to run
number_of_runs = 100

# Prepare combinations of armor and environment
armor_types = list(config['armor_profiles'].keys())
environments = list(config['threat_probs'].keys())
combinations = list(itertools.product(armor_types, environments))

output = Output()

def run_all_combinations(_):
    output.clear_output()
    blue_stock = 10
    red_stock = 40
    direction_deviation = 10

    results = []
    with output:
        # Wrap the outer loop with tqdm to show progress.
        for armor, env in tqdm(combinations, desc="Processing Combinations"):
            total_blue_remaining = 0
            total_red_remaining = 0
            total_effective_movement = 0
            total_blue_kills = 0
            total_blue_shots = 0
            total_red_kills = 0
            total_red_shots = 0

            # Wrap the inner loop with tqdm to show progress.
            for _ in tqdm(range(number_of_runs), desc=f"Running Simulations for {armor}-{env}", leave=False):
                params = {
                    "blue_stock": blue_stock,
                    "red_stock": red_stock,
                    "direction_deviation": direction_deviation,
                    "armor_type": armor,
                    "environment": env
                }
                result = run_simulation(params, full_log=False) # Set plot=False for multi-run

                total_blue_remaining += result['blue']['stock']
                total_red_remaining += result['red']['stock']

                # Accumulate the new metrics (adjust key names if necessary)
                total_effective_movement += result['blue']['patrol_distance']
                total_blue_kills += result['blue']['kills']
                total_blue_shots += result['blue']['shots']
                total_red_kills += result['red']['kills']
                total_red_shots += result['red']['shots']


            average_blue_remaining = total_blue_remaining / number_of_runs
            average_red_remaining = total_red_remaining / number_of_runs
            average_effective_movement = total_effective_movement / number_of_runs
            average_blue_lethality = total_blue_kills / total_blue_shots if total_blue_shots > 0 else 0
            average_red_lethality = total_red_kills / total_red_shots if total_red_shots > 0 else 0

            results.append({
                "Armor": armor,
                "Environment": env,
                "Average_Blue_Remaining": average_blue_remaining,
                "Average_red_Remaining": average_red_remaining,
                "Average_Effective_Movement": average_effective_movement,
                "Average_Blue_Lethality": average_blue_lethality,
                "Average_red_Lethality": average_red_lethality
            })

        # Plotting Blue Remaining
        fig1, ax1 = plt.subplots(figsize=(10, 6))
        for armor in armor_types:
            envs = [r['Environment'] for r in results if r['Armor'] == armor]
            blues = [r['Average_Blue_Remaining'] for r in results if r['Armor'] == armor]
            ax1.plot(envs, blues, marker='o', label=f'Armor: {armor}')
        ax1.set_title(f"Average Blue Remaining vs Environment ({number_of_runs} simulations per combination)")
        ax1.set_xlabel("Environment")
        ax1.set_ylabel(f"Average Blue Remaining (out of {blue_stock})")
        ax1.legend()
        plt.show()

        # Plotting Hostiles Remaining
        fig2, ax2 = plt.subplots(figsize=(10, 6))
        for armor in armor_types:
            envs = [r['Environment'] for r in results if r['Armor'] == armor]
            hostiles = [r['Average_red_Remaining'] for r in results if r['Armor'] == armor]
            ax2.plot(envs, hostiles, marker='o', label=f'Armor: {armor}')
        ax2.set_title(f"Average hostiles Remaining vs Environment ({number_of_runs} simulations per combination)")
        ax2.set_xlabel("Environment")
        ax2.set_ylabel(f"Average hostiles Remaining (out of {red_stock})")
        ax2.legend()
        plt.show()

        # You can add new plots for the new metrics here if desired.
        # For example, plotting Average Blue Lethality:
        fig3, ax3 = plt.subplots(figsize=(10, 6))
        for armor in armor_types:
            envs = [r['Environment'] for r in results if r['Armor'] == armor]
            blue_lethalities = [r['Average_Blue_Lethality'] for r in results if r['Armor'] == armor]
            ax3.plot(envs, blue_lethalities, marker='o', label=f'Armor: {armor}')
        ax3.set_title(f"Average Blue Lethality vs Environment ({number_of_runs} simulations per combination)")
        ax3.set_xlabel("Environment")
        ax3.set_ylabel("Average Blue Lethality")
        ax3.legend()
        plt.show()

        # Show results as a table
        df_results = pd.DataFrame(results)
        display(df_results) # Use display to show the DataFrame

run_button = Button(description=f"Run All Armor/Threat Combinations ({number_of_runs} times each)")
run_button.on_click(run_all_combinations)

VBox([run_button, output])
